## Analysis of results on DataCenterEnv

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style('darkgrid')

In [ ]:
def read_results_csv(filename):
    csv_path = os.path.join('logs', 'datacenter', filename)
    df = pd.read_csv(csv_path)
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
    df['setup'] = df[['train_carbon_yr', 'test_carbon_yr']].agg(tuple, axis=1)
    df = df[df['setup'].isin(((2019, 2021), (2021, 2021)))]
    df.rename(columns={'train_carbon_yr': 'trained on'}, inplace=True)
    return df

In [ ]:
dfs = []
filenames = ['all_MPC.csv', 'all_PPO.csv', 'all_SAC.csv', 'all_DDPG.csv']
for filename in filenames:
    df = read_results_csv(filename)
    if 'MPC' in filename:
        df = df[df['setup'] == (2021, 2021)]
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3), tight_layout=True)
sns.violinplot(df, x='model', y='reward', hue='trained on', ax=ax,
               hue_order=[2019, 2021])
ax.set(xlabel='Algorithm', ylabel='Daily Return ($)')
ax.axvline(0.5, linestyle=':', color='black')
fig.savefig('plots/datacenter/violinplot.png', bbox_inches='tight', dpi=300)